In [11]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [12]:
import os
import pandas as pd
import numpy as np
import joblib

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import CLEANED_DATA_DIR, TEST_PREDICTIONS_DIR
import project_utils.ml_utils as ml_utils

# Data pre-processing
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split

# Model
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV


Note: The model present in this notebook was trained without removing outliers. This decision was based on empirical observation: retaining outliers resulted in more robust models and improved generalization performance, whereas outlier removal consistently degraded validation stability and predictive accuracy.

# Stacking generalization

Stacking generalization is an ensemble learning methodology designed to improve predictive accuracy by systematically combining multiple base models. Rather than relying on a single learner, stacking trains a diverse collection of first-level models (often referred to as level-0 learners), each capturing different structural aspects or inductive biases present in the data. This method, rather than averaging or constructing a simple vote, combines the predictions produced by each base learner into a meta-model (often referred to as a second-level model), thus providing a principled framework for model combination [1].

The meta-model can then learn how to optimally weight, combine, or correct the systematic errors of the base learners. In practical implementations, the meta-learner is often regularized to prevent overfitting and to promote more stable combination rules.

In [13]:
df_train = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"), index_col="carID"
)
X_test = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"), index_col="carID"
)
X = df_train.drop("price", axis=1)
y = df_train["price"]
# Performing the holdout methods
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

This ensemble utilized the holdout method rather than cross-validation for several reasons. First, the base models incorporated into the ensemble were already trained using cross-validation, ensuring their individual robustness. Second, given the dataset size of approximately 70,000 observations, a 10% validation split provides sufficient data for reliable performance assessment without compromising the stacking ensemble's generalization capability. Finally, the holdout method significantly reduces computational costs compared to cross-validation, which would require nested cross-validation across both the meta-regressor (10 folds) and the stacking ensemble itself (10 folds), resulting in excessively expensive computation times.

In [14]:
MODELS_DIR = os.path.join(PROJ_ROOT, "src", "models")

# Decision tree models
hgbr_model = joblib.load(os.path.join(MODELS_DIR, "HGBR_model.joblib"))
etr_model = joblib.load(os.path.join(MODELS_DIR, "ETR_model.joblib"))
# KNN baggging
bagging_regressor = joblib.load(os.path.join(MODELS_DIR, "KNN_model.joblib"))

estimators = [
    ("hgbr", hgbr_model),
    ("etr", etr_model),
    ("bg", bagging_regressor),
]

# --- RidgeCV meta-learner ---
ridge_meta = RidgeCV(
    alphas= np.logspace(-4, 4, 20),
    cv=10
)

stacking_ensemble = StackingRegressor(
    estimators=estimators, final_estimator=ridge_meta, cv=10, n_jobs=-1
)

reg = TransformedTargetRegressor(
    regressor=stacking_ensemble, func=np.log1p, inverse_func=np.expm1
)

reg.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"regressor regressor: object, default=NoneRegressor object such as derived from:class:`~sklearn.base.RegressorMixin`. This regressor willautomatically be cloned each time prior to fitting. If `regressor isNone`, :class:`~sklearn.linear_model.LinearRegression` is created and used.",StackingRegre... n_jobs=-1)
,"transformer transformer: object, default=NoneEstimator object such as derived from:class:`~sklearn.base.TransformerMixin`. Cannot be set at the same timeas `func` and `inverse_func`. If `transformer is None` as well as`func` and `inverse_func`, the transformer will be an identitytransformer. Note that the transformer will be cloned during fitting.Also, the transformer is restricting `y` to be a numpy array.",None
,"func func: function, default=NoneFunction to apply to `y` before passing to :meth:`fit`. Cannot be setat the same time as `transformer`. If `func is None`, the function used will bethe identity function. If `func` is set, `inverse_func` also needs to beprovided. The function needs to return a 2-dimensional array.",<ufunc 'log1p'>
,"inverse_func inverse_func: function, default=NoneFunction to apply to the prediction of the regressor. Cannot be set atthe same time as `transformer`. The inverse function is used to returnpredictions to the same space of the original training labels. If`inverse_func` is set, `func` also needs to be provided. The inversefunction needs to return a 2-dimensional array.",<ufunc 'expm1'>
,"check_inverse check_inverse: bool, default=TrueWhether to check that `transform` followed by `inverse_transform`or `func` followed by `inverse_func` leads to the original targets.",True
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('drop_filtered_features', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('group_imputer', ...), ('column_transformer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to t

In [15]:
# Compute evaluation metrics for the training and validation sets
# Training set
train_predictions = reg.predict(X_train)
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(y_train, train_predictions, verbose=True)
# Validation set
validation_predictions = reg.predict(X_val)
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(y_val, validation_predictions, verbose=True)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


Training set metrics:
R²: 0.962, MAE: 1055.31, MAPE: 6.40%, RMSE: 1900.37
-------------------------------------------------------
Validation set metrics:
R²: 0.947, MAE: 1340.29, MAPE: 7.94%, RMSE: 2339.66
-------------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [16]:
# -- Export X_test predictions --#
pred_y_test = reg.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "ensemble_test_predictions.csv")
)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [17]:
joblib.dump(
    reg, os.path.join(PROJ_ROOT, "src", "models", "final_stacking_model.joblib")
)

['/Users/alexandre/Documents/GitHub/price_predictor/src/models/final_stacking_model.joblib']

## Model Performance Assessment and Final Remarks

### Overfitting Assessement and Practical interpretation

The final stacking ensemble delivers strong and robust predictive performance:

- **Training set**: R² ≈ 0.96, MAE ≈ 7% MAPE
- **Validation set**: R² ≈ 0.95, MAE ≈ 8% MAPE
- **External test set** (Kaggle platform): Lowest MAE among individual base models in the ensemble

The small gap between training and validation metrics indicates minimal overfitting. The slight increase in RMSE and MAPE on validation reflects both the presence of retained outliers and natural heterogeneity in car prices across models — suggesting that keeping realistic outliers during training actually improves generalization across the full market spectrum.

Practically, an average deviation of ~8% is excellent given the dataset's price range (€450–€16,000). This level of accuracy makes the model suitable for real-world deployment.

### Project Conclusion

This project developed a robust used-car price prediction pipeline through systematic data processing, feature engineering, model selection, and ensemble techniques.

Key insights:
- Simple baseline models capture linear trends effectively.
- Ensemble tree-based and ensemble methods better handle non-linear relationships and interactions.
- Stacking further improves performance by leveraging complementary strengths of diverse base learners.

The final stacking ensemble achieves high explanatory power (R² > 0.95), low relative error (~8% MAPE), and consistent generalization across training, validation, and external test data. These results demonstrate a favorable bias-variance trade-off and resilience to the inherent complexity of real-world car pricing.

Consequently, this stacking ensemble was selected as the core predictive engine for the web-based car price estimation application developed in the project's open-ended section.

## References

[1] D. H. Wolpert, “Stacked generalization,” Neural Networks, vol. 5, no. 2, pp. 241–259, Jan. 1992, doi: 10.1016/S0893-6080(05)80023-1.